In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In this notebook, we look at different aspects of data from various input files. This is work in progress. I will keep updating the notebook.

Feel free to suggest improvements

In [ ]:
stock_price=pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv")

In [ ]:
stock_price.head()

Check Shape of stock_price.

In [ ]:
stock_price.shape

This file has 2332531 rows and 12 columns

Check the range of dates for which we have data

In [ ]:
print('Oldest date:',stock_price['Date'].min())
print('Most recent date:',stock_price['Date'].max())

Check null values

In [ ]:
stock_price.isnull().sum()

ExpectedDividend has maximum number of null values. 

In [ ]:
stock_price[stock_price['Open'].isnull()].isnull().sum()

There are some cases where target is missing. Let us see these records.

In [ ]:
stock_price[stock_price['Target'].isnull()]

In [ ]:
stock_price[stock_price['Volume']==0]

As we can see, when volume is zero open,close etc values are also zero. Why are there no values for features or why is the volume 0 ? Is is because that there were trading holidays? Let us see. We should add few more date related fetaures to be able to do this.

In [ ]:
stock_price['Day of Week']=pd.to_datetime(stock_price['Date']).dt.dayofweek

In [ ]:
stock_price[stock_price['Volume']==0]['Day of Week'].value_counts()

As we can see we have data only for tarding days, i.e from Monday through Friday. We do not have Saturday & Sunday in data.So, let us us forward fill option of pandas fillna method to impute these values.

Most of the columns in the stock_prices are self explanatory. But, few columns like Adjustment Factor and Supervised Flag are not clear , so let us see what they have.

Adjustment Factor refers to multiplication factor to pice to account for bonus or splits etc.

Supervisor Flag referes to stocks that are under supervison.

In [ ]:
stock_price['AdjustmentFactor'].value_counts()

If value is more than 1 , its a bonus issue , if it is less than one its a stock split. So, we have create two additional columns with categorical data to indicate bonus issue and split. 

In [ ]:
stock_price[stock_price['AdjustmentFactor']==0.5]

In [ ]:
stock_price['SupervisionFlag'].value_counts()

When this flag changes from False to True, the impact on Target should be negative and when it turns from True to False, the impact on Target should be positive. We can have additional columns to indicate this transition.

In [ ]:
# impute missing values
stock_price.fillna(method='ffill',inplace=True)
#create column to indicate bonus
stock_price['bonus']=stock_price['AdjustmentFactor'].apply(lambda x:1 if x>1 else 0)
stock_price['split']=stock_price['AdjustmentFactor'].apply(lambda x:1 if x<1 else 0)

In [ ]:
stock_price=stock_price.sort_values(['SecuritiesCode','Date'])

In [ ]:
lst_of_securities=list(stock_price['SecuritiesCode'].unique())

In [ ]:
def flag_change(securityId):
    tdf=stock_price[stock_price['SecuritiesCode']==securityId]
    tdf['Flag Change']=tdf['SupervisionFlag'].diff()
    return tdf

In [ ]:
df=pd.DataFrame()
for i in lst_of_securities:
    df=df.append(flag_change(i))

In [ ]:
stock_price=df.sort_index()
del df

In [ ]:
stock_price['SupervisionFlag']=stock_price['SupervisionFlag'].apply(lambda x:1 if str(x)=='False' else 0)
stock_price['Flag Change']=stock_price['Flag Change'].apply(lambda x:1 if str(x)=='False' else 0)

We are almost done with our core data file. Just a few things like imputing missing values in Expected Dividend and splitting date into year and month and day remaining now.

In [ ]:
stock_price['ExpectedDividend'].fillna(0,inplace=True)
stock_price['Year']=pd.to_datetime(stock_price['Date']).dt.year
stock_price['Month']=pd.to_datetime(stock_price['Date']).dt.month
stock_price['Day']=pd.to_datetime(stock_price['Date']).dt.day
del stock_price['Date']

In [ ]:
stock_price.head()